## Intro

In this PA, you will practice altering data frame objects in order to solve a riddle.

### Advice for troubleshooting your code

#### Check results often

If a chunk of code runs smoothly without giving you any error or warnings this does not necessarily mean it accomplished the desired task.

It is a good habit to check the results of your code every time you finish a task.

If you created or updated a data frame, make sure your edits did what you hoped.  Use the *Environment* or the `head()` function to investigate your changes.

#### Document your work

In the text before the code chunk, make sure to briefly state what the point of the chunk is.  This will remind later readers - which might be your future self! - what the desired output is.

Use comments in the code to give quick explanations of the goals of complex steps.

#### Two heads are better than one

It can be hard to spot bugs in code that you yourself wrote.t
(Have you ever sent a text message and not spotted an obvious  typo?  Same idea!)

Work with people around you if you can - if something goes wrong, ask a friend to take a peek at your code and see if any glaring errors (like *syntax error*) pop out.

#### Explain your code out loud

The best way to troubleshoot a sneaky bug is to explain out loud each step of your code, and what you hoped to accomplish.

If you are alone, try [Rubber Duck Debugging](https://en.wikipedia.org/wiki/Rubber_duck_debugging)!

#### Google is your friend

The whole of the internet is at your disposal!  Use it early, use it often.

Copy-paste the exact error message into Google. Chances are, somebody else had a similar problem and got a similar message.


Include package names in your search terms.  For example, "bar plot in plotnine with pandas data frame" is a better search than "bar plot in python".


## Part One: Data import and cleaning

This section will walk you through some steps to clean up today's dataset, so that you can use it more easily in Part Two.  

First, we declare our package dependencies and load the data.

(Note that the data loading function `read_csv` will give you an outpouring of helpful information about the dataset.  If you do not see the word "error", there is nothing to be concerned about.)


In [1]:
import numpy as np
import pandas as pd

colleges = pd.read_csv("https://www.dropbox.com/s/bt5hvctdevhbq6j/colleges.csv?dl=1")

The code chunks in this section will cause an error and/or do the desired task incorrectly.  (Even the chunks that run without error are not correct!) You will need to find the mistake, and correct it, to complete the intended action.

None of the *text* is incorrect, only the *code* - your goal is to make the code do exactly what the documentation claims it will.

#### 1. Narrow down the columns to variables of interest

There are too many variables in this dataset.  We don't need all of them. We will limit our dataset down to only:

Name of the institution
City, State, and ZIP code of the institution
The Admissions Rate
The average SAT score
The number of undergraduate students
The in and out of state tuitions
Whether the school is public or private
The "REGION" variable.


In [2]:
colleges_clean = colleges[['INSTNM', 'CITY', 'STABBR', 'ZIP', 'CONTROL', 'ADM_RATE', 'SAT_AVG', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'UGDS', 'REGION']]

In [3]:
colleges_clean.columns

Index(['INSTNM', 'CITY', 'STABBR', 'ZIP', 'CONTROL', 'ADM_RATE', 'SAT_AVG',
       'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'UGDS', 'REGION'],
      dtype='object')

#### 2. Remove observations that aren't needed

Drop the schools that are "private for-profit" (category 3).  We will not need these for the puzzle.


In [4]:
#colleges_clean = colleges_clean[(colleges_clean['CONTROL'] == 1) | (colleges_clean['CONTROL'] == 2) ]
colleges_clean = colleges_clean[colleges_clean['CONTROL'] != 3]
#colleges_clean.describe

### 3. Adjust variable types: Numeric

Some of the columns of the dataset were automatically understood by the computer to be categorical, due to missing data appearing like strings.

Adjust the appropriate columns to be numeric.


In [5]:
colleges_clean['TUITIONFEE_IN'] = colleges_clean['TUITIONFEE_IN'].astype('float')
colleges_clean['TUITIONFEE_OUT'] = colleges_clean['TUITIONFEE_OUT'].astype('float')
colleges_clean['SAT_AVG'] = colleges_clean['SAT_AVG'].astype('float')
colleges_clean['ADM_RATE'] = colleges_clean['ADM_RATE'].astype('float')

In [6]:
colleges_clean.dtypes

INSTNM             object
CITY               object
STABBR             object
ZIP                object
CONTROL             int64
ADM_RATE          float64
SAT_AVG           float64
TUITIONFEE_IN     float64
TUITIONFEE_OUT    float64
UGDS              float64
REGION              int64
dtype: object

### 4. Adjust variable types: Categorical

Similarly, columns of the dataset that are not numbers are automatically understood by the computer to be strings.

We know, however, that some variables are truly *categorical*, in that there are only a few categories (or "levels") that can be values of the column.

Adjust the appropriate columns to be understood as categorical.


In [7]:
colleges_clean['CONTROL']= colleges_clean['CONTROL'].astype('category')
colleges_clean['REGION']= colleges_clean['REGION'].astype('category')

In [8]:
colleges_clean.dtypes

INSTNM              object
CITY                object
STABBR              object
ZIP                 object
CONTROL           category
ADM_RATE           float64
SAT_AVG            float64
TUITIONFEE_IN      float64
TUITIONFEE_OUT     float64
UGDS               float64
REGION            category
dtype: object

#### 5. Calculate new variables

Create a new variable called TUITION_DIFF which contains the difference between in and out of state costs.


In [9]:
colleges_clean['TUITION_DIFF'] = colleges_clean['TUITIONFEE_OUT']-colleges_clean['TUITIONFEE_IN']
colleges_clean

,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,TUITION_DIFF
0,Alabama A & M University,Normal,AL,35762,1,0.9027,929.0,9857.0,18236.0,4824.0,5,8379.0
1,University of Alabama at Birmingham,Birmingham,AL,35294-0110,1,0.9181,1195.0,8328.0,19032.0,12866.0,5,10704.0
2,Amridge University,Montgomery,AL,36117-3553,2,NaN,NaN,6900.0,6900.0,322.0,5,0.0
3,University of Alabama in Huntsville,Huntsville,AL,35899,1,0.8123,1322.0,10280.0,21480.0,6917.0,5,11200.0
4,Alabama State University,Montgomery,AL,36104-0271,1,0.9787,935.0,11068.0,19396.0,4189.0,5,8328.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7050,Arizona State University at Tucson,Tucson,AZ,857454284,1,NaN,NaN,NaN,NaN,NaN,6,NaN
7051,Arizona State University - ASU Colleges at Lak...,Lake Havasu City,AZ,864036877,1,NaN,NaN,6904.0,10106.0,NaN,6,3202.0
7052,Arizona State University at The Gila Valley,Thatcher,AZ,855525545,1,NaN,NaN,NaN,NaN,NaN,6,NaN
7053,Arizona State University at Yuma,Yuma,AZ,853656900,1,NaN,NaN,NaN,NaN,NaN,6,NaN


#### 6. Deal with missing data

Drop all the rows with missing data.

(It is often a terrible idea to simply drop all rows with *any* missing data!  Usually, even if *some* of the information is missing, we don't want to throw out the entire row.  In this example, however, we'll be lazy.)


In [10]:
colleges_clean = colleges_clean.dropna(axis = 0)

## Part Two:  Identify the mystery college

Wow!  Your best friend Ephelia has been accepted to the college of her dreams!  Unfortunately, Ephelia is a very mysterious person, and she won't tell you directly which college this is.  You'll have to use her clues to figure out which school is her dream school.

Clues:

1. This college is located in Region 1.

2. This college's admission rate is in the first quartile for the region.

3. This college charges the same for in- and out-of-state tuition.

4. The average SAT score of this college is an odd number.

5. This college is NOT in New Hampshire or in the city of Boston.

6. More than 3,000 people apply to this college every year.  (Hint: Make a new variable to compute how many people apply.  Assume the size of an incoming class is 1/4 of the total undergraduate population, and remember that not everyone who *applies* is *admitted*.)

7. Ephelia is not going to an Ivy League school.

8. Of the two options remaining at this step, Ephelia will attend the cheaper one.


In [11]:
#dataframe with only region 1
colleges_region1 = (colleges_clean[colleges_clean["REGION"] == 1]).reset_index(drop=True)
colleges_region1

,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,TUITION_DIFF
0,Albertus Magnus College,New Haven,CT,06511-1189,2,0.6388,930.0,31140.0,31140.0,1122.0,1,0.0
1,University of Bridgeport,Bridgeport,CT,06604-5620,2,0.5389,969.0,32250.0,32250.0,3129.0,1,0.0
2,Central Connecticut State University,New Britain,CT,6050,1,0.6676,1071.0,10225.0,22914.0,9317.0,1,12689.0
3,University of Connecticut,Storrs,CT,6269,1,0.4784,1313.0,14880.0,36948.0,18930.0,1,22068.0
4,University of Hartford,West Hartford,CT,06117-1599,2,0.8089,1140.0,38910.0,38910.0,4835.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
84,University of Connecticut-Avery Point,Groton,CT,06340-6097,1,0.9416,1111.0,12838.0,34906.0,619.0,1,22068.0
85,University of Connecticut-Stamford,Stamford,CT,06901-2315,1,0.8232,1085.0,12848.0,34916.0,1670.0,1,22068.0
86,Franklin W Olin College of Engineering,Needham,MA,02492-1200,2,0.1337,1522.0,51936.0,51936.0,345.0,1,0.0
87,University of Connecticut-Hartford Campus,Hartford,CT,6103,1,0.9597,1101.0,12828.0,34896.0,1244.0,1,22068.0


In [12]:
# using ADM_RATE as the reference column to get first quartile becuase REGION is not numeric
colleges_region1["ADM_RATE"] = pd.to_numeric(colleges_region1["ADM_RATE"], errors="coerce") # errors="coerce" means missing value = NaN instead of showing errors

Q1_rate = colleges_region1["ADM_RATE"
].quantile(0.25)

# Filter rows where ADM_RATE <= first quartile
first_quartile_schools = (colleges_region1[
    colleges_region1["ADM_RATE"] <= Q1_rate]
).reset_index(drop=True)

first_quartile_schools

,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,TUITION_DIFF
0,University of Connecticut,Storrs,CT,6269,1,0.4784,1313.0,14880.0,36948.0,18930.0,1,22068.0
1,Trinity College,Hartford,CT,06106-3100,2,0.3387,1370.0,54770.0,54770.0,2174.0,1,0.0
2,Yale University,New Haven,CT,6520,2,0.0693,1520.0,51400.0,51400.0,5742.0,1,0.0
3,Colby College,Waterville,ME,04901-8840,2,0.1579,1444.0,53120.0,53120.0,1917.0,1,0.0
4,Amherst College,Amherst,MA,01002-5000,2,0.1290,1497.0,54310.0,54310.0,1835.0,1,0.0
5,Babson College,Babson Park,MA,02457-0310,2,0.2418,1347.0,49664.0,49664.0,2342.0,1,0.0
6,Bentley University,Waltham,MA,02452-4705,2,0.4415,1319.0,48000.0,48000.0,4203.0,1,0.0
7,Boston College,Chestnut Hill,MA,2467,2,0.3241,1434.0,53346.0,53346.0,9636.0,1,0.0
8,Boston University,Boston,MA,2215,2,0.2511,1401.0,52082.0,52082.0,16716.0,1,0.0
9,Brandeis University,Waltham,MA,02454-9110,2,0.3422,1403.0,53537.0,53537.0,3624.0,1,0.0


In [13]:
#filter where schools' instate and out of state tuition are same
private_quartile_schools = first_quartile_schools[
    first_quartile_schools["TUITIONFEE_IN"] == first_quartile_schools["TUITIONFEE_OUT"]
].reset_index(drop=True)

private_quartile_schools

,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,TUITION_DIFF
0,Trinity College,Hartford,CT,06106-3100,2,0.3387,1370.0,54770.0,54770.0,2174.0,1,0.0
1,Yale University,New Haven,CT,6520,2,0.0693,1520.0,51400.0,51400.0,5742.0,1,0.0
2,Colby College,Waterville,ME,04901-8840,2,0.1579,1444.0,53120.0,53120.0,1917.0,1,0.0
3,Amherst College,Amherst,MA,01002-5000,2,0.1290,1497.0,54310.0,54310.0,1835.0,1,0.0
4,Babson College,Babson Park,MA,02457-0310,2,0.2418,1347.0,49664.0,49664.0,2342.0,1,0.0
5,Bentley University,Waltham,MA,02452-4705,2,0.4415,1319.0,48000.0,48000.0,4203.0,1,0.0
6,Boston College,Chestnut Hill,MA,2467,2,0.3241,1434.0,53346.0,53346.0,9636.0,1,0.0
7,Boston University,Boston,MA,2215,2,0.2511,1401.0,52082.0,52082.0,16716.0,1,0.0
8,Brandeis University,Waltham,MA,02454-9110,2,0.3422,1403.0,53537.0,53537.0,3624.0,1,0.0
9,Emerson College,Boston,MA,02116-4624,2,0.4606,1299.0,44832.0,44832.0,3799.0,1,0.0


In [14]:
with_odd_SAT = private_quartile_schools[private_quartile_schools["SAT_AVG"] % 2 == 1]
with_odd_SAT

,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,TUITION_DIFF
3,Amherst College,Amherst,MA,01002-5000,2,0.1290,1497.0,54310.0,54310.0,1835.0,1,0.0
4,Babson College,Babson Park,MA,02457-0310,2,0.2418,1347.0,49664.0,49664.0,2342.0,1,0.0
5,Bentley University,Waltham,MA,02452-4705,2,0.4415,1319.0,48000.0,48000.0,4203.0,1,0.0
7,Boston University,Boston,MA,2215,2,0.2511,1401.0,52082.0,52082.0,16716.0,1,0.0
8,Brandeis University,Waltham,MA,02454-9110,2,0.3422,1403.0,53537.0,53537.0,3624.0,1,0.0
9,Emerson College,Boston,MA,02116-4624,2,0.4606,1299.0,44832.0,44832.0,3799.0,1,0.0
10,Harvard University,Cambridge,MA,2138,2,0.0516,1523.0,48949.0,48949.0,7532.0,1,0.0
16,Williams College,Williamstown,MA,1267,2,0.1458,1493.0,53550.0,53550.0,2033.0,1,0.0
17,Dartmouth College,Hanover,NH,03755-3529,2,0.1045,1483.0,53368.0,53368.0,4315.0,1,0.0


In [15]:
#make a copy of dataframe to filter from 2 columns, makes it less messy, and we get to name the dataframe according to the current action
above_3000 = with_odd_SAT.copy()

above_3000 = with_odd_SAT[~((with_odd_SAT['CITY']== 'Boston') | (with_odd_SAT['STABBR'] == 'NH'))]

In [16]:
above_3000['freshmen'] = above_3000['UGDS'] / 4
above_3000['applicants'] = above_3000['freshmen'] / above_3000['ADM_RATE']
above_3000 = above_3000[above_3000['applicants'] > 3000]
above_3000

<positron-console-cell-16>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<positron-console-cell-16>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,TUITION_DIFF,freshmen,applicants
3,Amherst College,Amherst,MA,01002-5000,2,0.1290,1497.0,54310.0,54310.0,1835.0,1,0.0,458.75,3556.201550
10,Harvard University,Cambridge,MA,2138,2,0.0516,1523.0,48949.0,48949.0,7532.0,1,0.0,1883.00,36492.248062
16,Williams College,Williamstown,MA,1267,2,0.1458,1493.0,53550.0,53550.0,2033.0,1,0.0,508.25,3485.939643


In [17]:
not_ivy = above_3000.copy()
#create a list of all ivy league universities
ivy_league =[ 
    'Harvard University',
     'Brown University',
      'Columbia University', 
      'Cornell University', 
      'Dartmouth College',
      'University of Pennsylvania', 
      'Princeton University', 
      'Yale University']

not_ivy = not_ivy[~(not_ivy["INSTNM"].isin(ivy_league))].reset_index(drop=True)
not_ivy

,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,TUITION_DIFF,freshmen,applicants
0,Amherst College,Amherst,MA,01002-5000,2,0.1290,1497.0,54310.0,54310.0,1835.0,1,0.0,458.75,3556.201550
1,Williams College,Williamstown,MA,1267,2,0.1458,1493.0,53550.0,53550.0,2033.0,1,0.0,508.25,3485.939643


In [18]:
cheaper_school = not_ivy.nsmallest(1, "TUITIONFEE_IN")
cheaper_school

,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,TUITION_DIFF,freshmen,applicants
1,Williams College,Williamstown,MA,1267,2,0.1458,1493.0,53550.0,53550.0,2033.0,1,0.0,508.25,3485.939643


Therefore, Ephelia is going to Williams College
